# 서울시 미용실 데이터 - NAVER API 데이터 전처리
## 작성자(최종수정일) : 서동원(2020.04.11)

In [33]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import re
import requests
import time
pd.set_option('display.max_columns',50) # DataFrame truncation 없이 보기
pd.set_option('display.max_rows',100)
from tqdm import tqdm

## 네이버에 검색 작업 완료후 저장한 데이터

In [4]:
tmp_naver_1 = pd.read_csv('./naver_api_seoul_salon_data/naver_api_seoul_salon_data_2020-04-09(1-3200).csv', index_col=0)
tmp_naver_2 = pd.read_csv('./naver_api_seoul_salon_data/naver_api_seoul_salon_data_2020-04-09(3201-6400)(1).csv', index_col=0)
tmp_naver_3 = pd.read_csv('./naver_api_seoul_salon_data/naver_api_seoul_salon_data_2020-04-09(3201-6400)(2).csv', index_col=0)
tmp_naver_4 = pd.read_csv('./naver_api_seoul_salon_data/naver_api_seoul_salon_data_2020-04-09(6401-9600).csv', index_col=0)
tmp_naver_5 = pd.read_csv('./naver_api_seoul_salon_data/naver_api_seoul_salon_data_2020-04-09(9601-12800).csv', index_col=0)
tmp_naver_6 = pd.read_csv('./naver_api_seoul_salon_data/naver_api_seoul_salon_data_2020-04-09(12801-15903).csv', index_col=0)

In [5]:
print('1 - 3,200\t:', len(tmp_naver_1), '개')
print('3,201 - 6,400(1):', len(tmp_naver_2), '개')
print('3,201 - 6,400(2):', len(tmp_naver_3), '개')
print('6,401 - 9,600\t:', len(tmp_naver_4), '개')
print('9,601 - 12,800\t:', len(tmp_naver_5), '개')
print('12,801 - 15,907\t:', len(tmp_naver_6), '개')
print('total count\t:', len(tmp_naver_1)+len(tmp_naver_2)+len(tmp_naver_3)+len(tmp_naver_4)+len(tmp_naver_5)+len(tmp_naver_6), '개')

1 - 3,200	: 45666 개
3,201 - 6,400(1): 18304 개
3,201 - 6,400(2): 29720 개
6,401 - 9,600	: 46638 개
9,601 - 12,800	: 49459 개
12,801 - 15,907	: 48548 개
total count	: 238335 개


In [6]:
tmp_naver_1.head(12)

,업소명,소재지전화번호,소재지도로명,소재지지번,카테고리,검색어,검색결과수
0,조희헤어미용실,02-573-6218,서울특별시 강남구 압구정로30길 17 이소니프라자빌딩 B1,서울특별시 강남구 신사동 609,"생활,편의>미용실",서울 조희미용실 미용실,11
1,조희미용실,02-447-6748,서울특별시 광진구 뚝섬로38길 14,서울특별시 광진구 자양동 460-4,"생활,편의>미용실",서울 조희미용실 미용실,11
2,조희미용실,02-555-0157,서울특별시 강남구 남부순환로 2917 1층 111-가,서울특별시 강남구 대치동 626,"생활,편의>미용실",서울 조희미용실 미용실,11
3,조희미용실,02-447-0658,서울특별시 광진구 뚝섬로52나길 25,서울특별시 광진구 자양동 651-3,"생활,편의>미용실",서울 조희미용실 미용실,11
4,조희미용실,02-403-9589,서울특별시 송파구 송파대로18길 16,서울특별시 송파구 문정1동 65,"생활,편의>미용실",서울 조희미용실 미용실,11
5,조희미용실,02-716-9398,서울특별시 마포구 백범로1길 8,서울특별시 마포구 노고산동 31-77,"생활,편의>미용실",서울 조희미용실 미용실,11
6,조희미용실,NaN,서울특별시 서초구 동광로12가길 21,서울특별시 서초구 방배동 1448,"생활,편의>미용실",서울 조희미용실 미용실,11
7,조희미용실,02-987-7370,서울특별시 강북구 솔샘로 263-1,서울특별시 강북구 미아동 705-3,"생활,편의>미용실",서울 조희미용실 미용실,11
8,조희미용실,02-713-2986,서울특별시 마포구 마포대로16길 25-1,서울특별시 마포구 공덕동 91-17,"생활,편의>미용실",서울 조희미용실 미용실,11
9,조희미용실,02-762-0886,서울특별시 성북구 성북로 65,서울특별시 성북구 성북동 133-41,"생활,편의>미용실",서울 조희미용실 미용실,11


In [7]:
naver_search_raw = pd.concat([tmp_naver_1, tmp_naver_2, tmp_naver_3, tmp_naver_4, tmp_naver_5, tmp_naver_6], ignore_index=True)
pre_len = len(naver_search_raw)
naver_search_raw.tail()

,업소명,소재지전화번호,소재지도로명,소재지지번,카테고리,검색어,검색결과수
238330,우주이발관,NaN,서울특별시 동대문구 사가정로13길 27,서울특별시 동대문구 전농동 60-15,"생활,편의>이용원",서울 우주헤어 미용실,6
238331,우주이용원,NaN,서울특별시 용산구 우사단로10길 127,서울특별시 용산구 한남동 768-16,"생활,편의>이용원",서울 우주헤어 미용실,6
238332,우주미용실,NaN,서울특별시 동대문구 전농로16길 33,서울특별시 동대문구 전농동 46-50,"생활,편의>미용실",서울 우주헤어 미용실,6
238333,민티헤어,02-877-1147,서울특별시 관악구 남부순환로 1913 우주빌딩,서울특별시 관악구 봉천동 1685-20,"생활,편의>미용실",서울 우주헤어 미용실,6
238334,승리미용실,NaN,서울특별시 영등포구 신길로41길 4,서울특별시 영등포구 신길동 224-25 우주빌라,"생활,편의>미용실",서울 우주헤어 미용실,6


In [8]:
naver_search_raw.drop_duplicates(inplace=True)
print('중복 제거 전 개수 :', pre_len)
print('중복 제거 후 개수 :', len(naver_search_raw))
pre_len = len(naver_search_raw)

중복 제거 전 개수 : 238335
중복 제거 후 개수 : 96682


In [9]:
naver_search_raw['카테고리'].unique()

array(['생활,편의>미용실', '기업>프랜차이즈본사', '미용>미용실', '생활,편의>이용원', '생활,편의>남성전문미용실',
       '생활,편의>미용', '미용>메이크업', '미용>머리염색', '메이크업>웨딩메이크업', '미용>네일아트,네일샵',
       '미용>두피,탈모관리', '미용>피부,체형관리', '소프트웨어개발>경영,고객관리', '반려동물>반려동물미용',
       '패션잡화>가발', '서비스,산업>건설업', '전문,기술서비스>인테리어디자인', '서비스,산업>기업',
       '쇼핑,유통>미용기기,재료', '협회,단체>화장품,미용', '쇼핑,유통>화장품,향수', '미용실>어린이미용실',
       '쇼핑,유통>휴대폰', '쇼핑,유통>가구,인테리어', '직업,기술학원>미용', '전문,기술서비스>경영컨설팅',
       '제조업>화장품제조', '청소>상업세탁물', '쇼핑,유통>남성정장', '협회,단체>창업,취업', '쇼핑,유통>주방가구',
       '부동산>중개업', '음식점>한식', '공중전화', '제조업>의류제작', '컴퓨터프로그래밍,정보서비스업>소프트웨어개발',
       '음식점>카페,디저트', '반려동물>애견용품', '반려동물>애견훈련', '쇼핑,유통>반찬가게',
       '하수,폐기물,환경>폐기물수집,처리업', '직업,기술학원>애견미용', '한식>찜닭', '건설업>건물,구축물해체공사',
       '제조업>간판제작', '술집>맥주,호프', '광고,마케팅>광고물작성', '생활,편의>공방', '광고,마케팅>광고대행',
       '전문,기술서비스>세무사', '쇼핑,유통>곡류', '서비스,산업>정보,컨텐츠제작', '쇼핑,유통>시장',
       '쇼핑,유통>종합도소매', '음식점>치킨,닭강정'], dtype=object)

In [10]:
naver_search_raw[naver_search_raw['카테고리'] == '미용>네일아트,네일샵']

,업소명,소재지전화번호,소재지도로명,소재지지번,카테고리,검색어,검색결과수
2045,위례 라곰네일,NaN,서울특별시 송파구 위례광장로 270 송파와이즈더샵 G동 2층 229호,서울특별시 송파구 장지동 895,"미용>네일아트,네일샵",서울 뷰티네일 미용실,113
2062,현아네일,NaN,서울특별시 강동구 명일로10길 9,서울특별시 강동구 둔촌동 39,"미용>네일아트,네일샵",서울 뷰티네일 미용실,113
72606,위례 라곰네일,NaN,서울특별시 송파구 위례광장로 270 송파와이즈더샵 G동 2층 229호,서울특별시 송파구 장지동 895,"미용>네일아트,네일샵",서울 더샵 미용실 미용실,13
175546,위례 라곰네일,NaN,서울특별시 송파구 위례광장로 270 송파와이즈더샵 G동 2층 229호,서울특별시 송파구 장지동 895,"미용>네일아트,네일샵",서울 와이즈살롱 미용실,12
178583,현아네일,NaN,서울특별시 강동구 명일로10길 9,서울특별시 강동구 둔촌동 39,"미용>네일아트,네일샵",서울 현아 미용실,6
203527,현아네일,NaN,서울특별시 강동구 명일로10길 9,서울특별시 강동구 둔촌동 39,"미용>네일아트,네일샵",서울 현아미용실 미용실,6
218975,위례 라곰네일,NaN,서울특별시 송파구 위례광장로 270 송파와이즈더샵 G동 2층 229호,서울특별시 송파구 장지동 895,"미용>네일아트,네일샵",서울 더샵 미용실,13


In [11]:
naver_search_raw[['업소명', '소재지전화번호','소재지도로명','소재지지번']].duplicated()

0         False
1         False
2         False
3         False
4         False
          ...  
238330     True
238331     True
238332     True
238333     True
238334     True
Length: 96682, dtype: bool

In [12]:
len(naver_search_raw[naver_search_raw[['업소명', '소재지전화번호','소재지도로명','소재지지번']].duplicated()])

78761

In [13]:
test = naver_search_raw[['업소명', '소재지전화번호','소재지도로명','소재지지번']].drop_duplicates()
len(test)

17921

In [14]:
test_merge = pd.merge(test, naver_search_raw)
test_merge

,업소명,소재지전화번호,소재지도로명,소재지지번,카테고리,검색어,검색결과수
0,조희헤어미용실,02-573-6218,서울특별시 강남구 압구정로30길 17 이소니프라자빌딩 B1,서울특별시 강남구 신사동 609,"생활,편의>미용실",서울 조희미용실 미용실,11
1,조희헤어미용실,02-573-6218,서울특별시 강남구 압구정로30길 17 이소니프라자빌딩 B1,서울특별시 강남구 신사동 609,"생활,편의>미용실",서울 명동미용실 미용실,81
2,조희헤어미용실,02-573-6218,서울특별시 강남구 압구정로30길 17 이소니프라자빌딩 B1,서울특별시 강남구 신사동 609,"생활,편의>미용실",서울 명동헤어샵 미용실,81
3,조희헤어미용실,02-573-6218,서울특별시 강남구 압구정로30길 17 이소니프라자빌딩 B1,서울특별시 강남구 신사동 609,"생활,편의>미용실",서울 조희헤어 미용실,20
4,조희헤어미용실,02-573-6218,서울특별시 강남구 압구정로30길 17 이소니프라자빌딩 B1,서울특별시 강남구 신사동 609,"생활,편의>미용실",서울 조희헤어샵 미용실,20
...,...,...,...,...,...,...,...
96677,광일이발관,NaN,서울특별시 중랑구 망우로36길 31,서울특별시 중랑구 상봉동 128-71,"생활,편의>이용원",서울 상봉동 미용실 미용실,107
96678,엄헤어샵,NaN,서울특별시 중랑구 상봉중앙로 51,서울특별시 중랑구 상봉동 216,"생활,편의>미용실",서울 상봉동 미용실 미용실,107
96679,엔트린헤어클럽,NaN,서울특별시 중랑구 봉우재로25길 30,서울특별시 중랑구 상봉동 111-80,"생활,편의>미용실",서울 상봉동 미용실 미용실,107
96680,미애킴미용실,NaN,서울특별시 중랑구 면목로91길 39,서울특별시 중랑구 상봉동 111-55,"생활,편의>미용실",서울 상봉동 미용실 미용실,107


In [15]:
naver_search_raw[['업소명', '소재지전화번','소재지도로명','소재지지번','검색어','검색결과수']].duplicated

KeyError: "['소재지전화번'] not in index"

In [ ]:
print(len(naver_search_raw[naver_search_raw['카테고리'] == '생활,편의>미용실']))
print(len(naver_search_raw[naver_search_raw['카테고리'] == '미용>미용실']))
print(len(naver_search_raw[naver_search_raw['카테고리'] == '생활,편의>이용원']))
print(len(naver_search_raw[naver_search_raw['카테고리'] == '생활,편의>미용']))
print(len(naver_search_raw[naver_search_raw['카테고리'] == '생활,편의>남성전문미용실']))
print(len(naver_search_raw[naver_search_raw['카테고리'] == '쇼핑,유통>미용기기,재료']))
print(len(naver_search_raw[naver_search_raw['카테고리'] == '미용실>어린이미용실']))

In [ ]:
naver_search_raw[naver_search_raw['카테고리'] == '생활,편의>미용실']

## 네이버에 검색 결과가 너무 많아 검색을 보류한 데이터

In [20]:
tmp_naver_rejected_1 = pd.read_csv('./naver_api_seoul_salon_data/naver_api_seoul_salon_rejected_data_2020-04-09(1-3200).csv', index_col=0)
tmp_naver_rejected_2 = pd.read_csv('./naver_api_seoul_salon_data/naver_api_seoul_salon_rejected_data_2020-04-09(3201-6400)(1).csv', index_col=0)
tmp_naver_rejected_3 = pd.read_csv('./naver_api_seoul_salon_data/naver_api_seoul_salon_rejected_data_2020-04-09(3201-6400)(2).csv', index_col=0)
tmp_naver_rejected_4 = pd.read_csv('./naver_api_seoul_salon_data/naver_api_seoul_salon_rejected_data_2020-04-09(6401-9600).csv', index_col=0)
tmp_naver_rejected_5 = pd.read_csv('./naver_api_seoul_salon_data/naver_api_seoul_salon_rejected_data_2020-04-09(9601-12800).csv', index_col=0)
tmp_naver_rejected_6 = pd.read_csv('./naver_api_seoul_salon_data/naver_api_seoul_salon_rejected_data_2020-04-09(12801-15903).csv', index_col=0)

In [21]:
print('1 - 3,200\t:', len(tmp_naver_rejected_1), '개')
print('3,201 - 6,400(1):', len(tmp_naver_rejected_2), '개')
print('3,201 - 6,400(2):', len(tmp_naver_rejected_3), '개')
print('6,401 - 9,600\t:', len(tmp_naver_rejected_4), '개')
print('9,601 - 12,800\t:', len(tmp_naver_rejected_6), '개')
print('12,801 - 15,907\t:', len(tmp_naver_rejected_6), '개')
print('total count\t:', len(tmp_naver_rejected_1)+len(tmp_naver_rejected_2)+len(tmp_naver_rejected_3)\
      +len(tmp_naver_rejected_4)+len(tmp_naver_rejected_5)+len(tmp_naver_rejected_6), '개')

1 - 3,200	: 28 개
3,201 - 6,400(1): 17 개
3,201 - 6,400(2): 25 개
6,401 - 9,600	: 24 개
9,601 - 12,800	: 38 개
12,801 - 15,907	: 38 개
total count	: 168 개


In [22]:
naver_search_rejected_raw = pd.concat([tmp_naver_rejected_1, tmp_naver_rejected_2, tmp_naver_rejected_3
                                       , tmp_naver_rejected_4, tmp_naver_rejected_5, tmp_naver_rejected_6], ignore_index=True)
pre_rejected = len(naver_search_rejected_raw)

In [23]:
naver_search_rejected_raw.drop_duplicates(inplace=True)
print('중복 제거 전 개수 :', pre_rejected)
print('중복 제거 후 개수 :', len(naver_search_rejected_raw))
pre_rejected = len(naver_search_rejected_raw)

중복 제거 전 개수 : 168
중복 제거 후 개수 : 168


In [34]:
naver_search_rejected_raw[100:]

,인덱스,지역,업소명,소재지도로명,검색결과수
100,10133,성북,서울,"서울특별시 성북구 삼양로8길 3, (길음동)",20191
101,10135,성북,머리방미용실,"서울특별시 성북구 종암로38길 15, (하월곡동)",18305
102,10428,성북,헤어매직,"서울특별시 성북구 오패산로 55, (하월곡동,장씨빌딩 202호)",698
103,10745,성북,머리헤어,"서울특별시 성북구 돌곶이로14길 25, (석관동)",18365
104,10846,성북,남성미용실,"서울특별시 성북구 돌곶이로 3, 1층 (석관동)",784
105,10864,송파,서울,"서울특별시 송파구 송파대로36길 7-38, (송파동)",20190
106,11272,송파,헤어아트,"서울특별시 송파구 거마로20길 39-10, 1층 (마천동)",18330
107,11404,송파,헤어＃,"서울특별시 송파구 올림픽로4길 17, (잠실동,지층)",19904
108,11434,송파,헤어스타일,"서울특별시 송파구 풍성로25길 24, (풍납동)",384
109,11435,송파,헤어매직,"서울특별시 송파구 올림픽로35길 124, (신천동, 장미아파트A상가1층105호)",698
